# ReAct agent with Wikipedia and calculator tool access

Trying to learn to make basic agents that can use tools. I found a YouTube video from June 2023 that used ReAct with Langchain to look stuff up with Wikipedia [(link)](https://www.youtube.com/watch?v=Eug2clsLtFs). However, the code used there doesn't work any more (all been deprecated) so I had to do most of this from scratch. I added in a calculator tool as well (which was a bit messy as I had to define a wrapper for the class so it would have a description attribute). Seems to work reasonably well now

## Importing libraries and setting up llm

In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMMathChain
from langchain.tools import BaseTool


## Set up logging with LangSmith

In [2]:
# LANGCHAIN_TRACING_V2="true"
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
# LANGCHAIN_PROJECT="langchain_react_wikipedia"

import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain_react_wikipedia"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"  #



# import os



# os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "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"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "langchain_react_wikipedia"

In [3]:
from langsmith import utils
utils.tracing_is_enabled()

True

## Initialise language model

In [4]:

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Defining wikipedia tool

In [5]:

# Set up the Wikipedia tool
api_wrapper = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)


## Defining calculator tool

In [6]:

# Create a proper tool wrapper by extending BaseTool
class MathToolWrapper(BaseTool):
    name: str = "Math Calculator"
    description: str = "A tool to perform mathematical calculations."
    _math_chain: LLMMathChain  # Private attribute

    def __init__(self, llm):
        super().__init__()
        self._math_chain = LLMMathChain(llm=llm)

    def _run(self, query: str):
        return self._math_chain.run(query)

    async def _arun(self, query: str):
        raise NotImplementedError("Async mode is not supported for this tool.")

# Initialize the math tool with the wrapper
math_tool = MathToolWrapper(llm=llm)


/var/folders/78/7l8366_n3pj6jn47gs4m1dkh0000gn/T/ipykernel_73375/621331216.py:9: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  self._math_chain = LLMMathChain(llm=llm)
/Users/nikmitchell/miniconda3/envs/langchain_react/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


## Defining the agent

In [7]:

tools = [wikipedia_tool, math_tool]

# Define the prompt template
prompt_template = PromptTemplate.from_template(
    '''Answer the following questions as best you can.
    You have access to the following tools:
    {tools}
    Use the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    Begin!
    Question: {input}
    Thought:{agent_scratchpad}'''
)

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template
)

# Set up the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


## Asking it questions

In [8]:

# Execute the agent with a specific query
# query = "How old is the president of the united states? Divide your answer by 3 and add 7.5"
query ="What is the first film that Russell Crowe won an Oscar for, and who directed that movie? When was that director born? Take your answer and square root it"
response = agent_executor.invoke({"input": query})
print(response)



> Entering new AgentExecutor chain...
To answer the question, I need to find out which film Russell Crowe won his first Oscar for, who directed that film, and when the director was born. After gathering that information, I will then take the square root of the birth year of the director.

Action: wikipedia  
Action Input: "Russell Crowe first Oscar win"  Page: Gladiator (2000 film)
Summary: Gladiator is a 2000 historical epic film directed by Ridley Scott and written by David Franzoni, John Logan, and William Nicholson from a story by Franzoni. It stars Russell Crowe, Joaquin Phoenix, Connie Nielsen, Oliver Reed, Derek Jacobi, Djimon Hounsou, and Richard Harris. Crowe portrays the Roman general Maximus Decimus Meridius, who is betrayed when Commodus, the ambitious son of Emperor Marcus Aurelius, murders his father and seizes the throne. Reduced to slavery, Maximus becomes a gladiator and rises through the ranks of the arena, determined to avenge the murders of his family and the empe

/var/folders/78/7l8366_n3pj6jn47gs4m1dkh0000gn/T/ipykernel_8345/621331216.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self._math_chain.run(query)


Answer: 44.01136216933077I now know the final answer.  
Final Answer: 44.01136216933077

> Finished chain.
{'input': 'What is the first film that Russell Crowe won an Oscar for, and who directed that movie? When was that director born? Take your answer and square root it', 'output': '44.01136216933077'}
